<a href="https://colab.research.google.com/github/jonahtan1995/Python-deciper/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install heapdict

In [12]:
import os
import sys
from google.colab import drive
import time
from IPython.display import clear_output

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the path to your notebook directory (adjust as per your structure)
my_path = '/content/drive/My Drive/Colab Notebooks/Homework1'

# Set environment variable for MY_PATH (optional)
os.environ['MY_PATH'] = my_path

# Copy files from Google Drive to Colab environment
!cp -r "$MY_PATH"/* .

# Add the path to your Python files to sys.path
sys.path.append(my_path)


from CampusEnv import CampusEnv
from Algorithms import *


DOWN = 0
RIGHT = 1
UP = 2
LEFT = 3

from collections import deque
from typing import Tuple, List, Set
from heapdict import heapdict

MAPS = {
    "4x4": ["SFFF",
            "FHFH",
            "FFFH",
            "HFFG"],
    "8x8": ["SFFFFFFF",
            "FFFFFTAL",
            "TFFHFFTF",
            "FPFFFHTF",
            "FAFHFPFF",
            "FHHFFFHF",
            "FHTFHFTL",
            "FLFHFFFG"],
}

env = CampusEnv(MAPS["8x8"])
env.reset()



def print_solution(actions,env: CampusEnv) -> None:
    env.reset()
    total_cost = 0
    print(env.render())
    print(f"Timestep: {1}")
    print(f"State: {env.get_state()}")
    print(f"Action: {None}")
    print(f"Cost: {0}")
    time.sleep(1)

    for i, action in enumerate(actions):
      state, cost, terminated = env.step(action)
      total_cost += cost
      clear_output(wait=True)

      print(env.render())
      print(f"Timestep: {i + 2}")
      print(f"State: {state}")
      print(f"Action: {action}")
      print(f"Cost: {cost}")
      print(f"Total cost: {total_cost}")

      time.sleep(1)

      if terminated is True:
        break

class Node:
    def __init__(self,state,action,cost,parent)-> None:
        self.state=state #position on the board
        self.action=action #the action that got us to here
        self.cost=cost # the price of the action
        self.parent=parent # wich node is the parnet


    def __hash__(self):
        return hash(self.state)

    def __eq__(self,other):
        return other.state==self.state

class Agent():
    def __init__(self) -> None:
        self.expanded=0
    def gen_solution(self,node:Node)->Tuple[list[int],int,int]:
        steps=[]
        sum_of_costs=0
        while node.parent:
            steps.append(node.action)
            sum_of_costs+=node.cost
            node=node.parent
        steps.reverse()
        return(list(steps),sum_of_costs,self.expanded)


class DFSGAgent(Agent):
    def __init__(self) -> None:
        super().__init__()


    def depth_first_search_graph(self,env)-> Tuple[list[int],int,int]:
        node=Node(env.get_initial_state(),None,0,None)
        self.expanded=1
        if(env.is_final_state(node.state)):
            return self.gen_solution(node)
        close_nodes=set()
        open_nodes=deque([node])

        return self.recursive_dfs_g(env,open_nodes,close_nodes)

    def recursive_dfs_g(self,env,open_nodes,close_nodes,)-> Tuple[List[int], int, int]:
        if not open_nodes:
            return None

        node=open_nodes.pop()
        close_nodes.add(node.state)

        if env.is_final_state(node.state):
            self.gen_solution(node)

        for action, successor in env.succ(node.state).items():
            if successor[0] is None:
                continue

            child=Node(successor[0],action,successor[1],node)

            if child.state not in close_nodes and child not in open_nodes:
                if(env.is_final_state(child.state)):
                    return self.gen_solution(child)
                open_nodes.append(child)
                self.expanded+=1
                res=self.recursive_dfs_g(env,open_nodes,close_nodes)
                if res:
                    return res
        return None


DFSG_agent = DFSGAgent()
actions, total_cost, expanded = DFSG_agent.depth_first_search_graph(env)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")

assert total_cost == 148.0, "Error in total cost returned"
assert expanded == 20, "Error in number of expanded nodes returned"


class UCSAgent(Agent):
    def __init__(self) -> None:
        super().__init__()

    def ucs_search(self,env)-> Tuple[List[int], int ,int]:
        node=Node(env.get_initial_state(),None,0,None)
        self.expanded=0
        if env.is_final_state(node.state):
            return self.gen_solution(node)
        close_nodes=set()
        open_nodes=heapdict()
        open_nodes[node]=(0,node.state)

        while open_nodes:
            node,val=open_nodes.popitem()
            close_nodes.add(node.state)
            if env.is_final_state(node.state):
                return self.gen_solution(node)

            self.expanded=self.expanded+1

            for action,successor in env.succ(node.state).items():
                if successor[0] is None:
                    continue
                new_cost=val[0]+successor[1]
                child=Node(successor[0],action,successor[1],node)

                if child.state  not in close_nodes and child not in open_nodes:
                    open_nodes[child]=(new_cost,successor[0])
                elif child is open_nodes and open_nodes.get(child)[0]>new_cost:
                    open_nodes[child]=(0,successor[0])
                    open_nodes.popitem()
                    open_nodes[child]=(new_cost,successor[0])



UCS_agent = UCSAgent()
actions, total_cost, expanded = UCS_agent.ucs_search(env)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")


Mounted at /content/drive
Total_cost: 93.0
Expanded: 47
Actions: [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
